In [3]:
!pip install opencv-python matplotlib diffusers transformers torch torchvision 

  Using cached diffusers-0.31.0-py3-none-any.whl.metadata (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 57.2 MB/s eta 0:00:00:00:01


In [34]:
!pip install rembg opencv-python-headless diffusers transformers torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.4 MB/s eta 0:00:00


In [36]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 98.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.1 MB/s eta 0:00:00


In [46]:
import torch
import os
from PIL import Image, ImageFilter
from rembg import remove  # Background removal
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel


# Step 1: Load Stable Diffusion ControlNet Models
def load_models():
    print("Loading Stable Diffusion ControlNet models...")
    controlnet = ControlNetModel.from_pretrained(
        "lllyasviel/control_v11p_sd15_canny", torch_dtype=torch.float16
    )
    pipeline = StableDiffusionControlNetPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
    )
    pipeline.to("cuda")
    return pipeline


# Step 2: Generate Product Mask Using rembg
def generate_product_mask(product_image_path, debug_dir):
    print("Generating product mask using rembg...")
    product_image = Image.open(product_image_path).convert("RGBA")
    mask = remove(product_image, only_mask=True)
    mask = mask.convert("L")
    mask_debug_path = os.path.join(debug_dir, "product_mask_debug.png")
    mask.save(mask_debug_path)
    print(f"Saved product mask to '{mask_debug_path}'")
    return mask


# Step 3: Add Realistic Shadows
def add_shadow(mask, shadow_offset=(15, 15), shadow_opacity=80, shadow_blur=25):
    shadow = mask.filter(ImageFilter.GaussianBlur(shadow_blur))
    shadow_layer = Image.new("RGBA", mask.size, (0, 0, 0, 0))
    shadow_layer.paste((0, 0, 0, shadow_opacity), shadow_offset, mask=shadow)
    return shadow_layer


# Step 4: Blend Product Image with Background
def blend_with_background(product_image_path, mask, background, output_dir, shadow_offset=(15, 15)):
    print("Blending product with background...")
    product_image = Image.open(product_image_path).convert("RGBA")
    background = background.resize((1024, 1024), Image.Resampling.LANCZOS)
    product_resized = product_image.resize(background.size, Image.Resampling.LANCZOS)
    mask_resized = mask.resize(background.size, Image.Resampling.LANCZOS)
    shadow_layer = add_shadow(mask_resized, shadow_offset)
    transparent_layer = Image.new("RGBA", background.size, (0, 0, 0, 0))
    transparent_layer.paste(shadow_layer, (0, 0), mask=mask_resized)
    transparent_layer.paste(product_resized, (0, 0), mask=mask_resized)
    final_image = Image.alpha_composite(background.convert("RGBA"), transparent_layer)
    return final_image.convert("RGB")


# Step 5: Generate Backgrounds Using ControlNet
def generate_backgrounds(controlnet_pipeline, product_image_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    debug_dir = os.path.join(output_dir, "debug")
    if not os.path.exists(debug_dir):
        os.makedirs(debug_dir)

    mask = generate_product_mask(product_image_path, debug_dir)
    prompts = {
        "studio": "Vibrant yellow gradient studio background, smooth lighting, reflective ground surface.",
        "urban": "Dark moody urban alley with wet pavement, soft reflections.",
        "solid": "Solid matte grey background with subtle lighting.",
    }
    negative_prompt = "Shoes, people, text, artifacts, noise, distortions"

    for key, prompt in prompts.items():
        print(f"Generating '{key}' background...")
        generated_image = controlnet_pipeline(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=50,
            guidance_scale=7.5,
            image=mask
        ).images[0]
        background_path = os.path.join(debug_dir, f"{key}_background_debug.png")
        generated_image.save(background_path)
        print(f"Saved generated background to '{background_path}'")

        final_image = blend_with_background(product_image_path, mask, generated_image, output_dir)
        output_path = os.path.join(output_dir, f"{key}_final.png")
        final_image.save(output_path)
        print(f"Saved final image to '{output_path}'")


# Main Execution
if __name__ == "__main__":
    product_image_path = "/kaggle/input/product/Jordan1.jpeg"
    output_dir = "/kaggle/working/"

    print("Loading models...")
    controlnet_pipeline = load_models()

    print("Generating backgrounds...")
    generate_backgrounds(controlnet_pipeline, product_image_path, output_dir)

    print("Done. Check the 'output' folder.")


Loading models...
Loading Stable Diffusion ControlNet models...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Generating backgrounds...
Generating product mask using rembg...
Saved product mask to '/kaggle/working/debug/product_mask_debug.png'
Generating 'studio' background...


  0%|          | 0/50 [00:00<?, ?it/s]

Saved generated background to '/kaggle/working/debug/studio_background_debug.png'
Blending product with background...
Saved final image to '/kaggle/working/studio_final.png'
Generating 'urban' background...


  0%|          | 0/50 [00:00<?, ?it/s]

Saved generated background to '/kaggle/working/debug/urban_background_debug.png'
Blending product with background...
Saved final image to '/kaggle/working/urban_final.png'
Generating 'solid' background...


  0%|          | 0/50 [00:00<?, ?it/s]

Saved generated background to '/kaggle/working/debug/solid_background_debug.png'
Blending product with background...
Saved final image to '/kaggle/working/solid_final.png'
Done. Check the 'output' folder.
